In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
MAX_DATA=55000
def get_batch(n):
    ni=np.random.randint(0,MAX_DATA,n)
    x = mnist.train.images[ni] 
    y = mnist.train.labels[ni]
    return x,y

In [4]:
class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__(
            c1=L.Convolution2D(1, 16, ksize=3, pad=1,),
            c2=L.Convolution2D(16, 64, ksize=3, pad=1),
            c3=L.Convolution2D(64, 256, ksize=3, pad=1),
            l1 = L.Linear(4*4*256,512),
            l2 = L.Linear(512,10),
        )
        
    def __call__(self,x,t):
        return F.softmax_cross_entropy(self.predict(x),t)

    def  predict(self,x):
        h1 = F.max_pooling_2d( F.relu(self.c1(x)),2)
        h2 = F.max_pooling_2d(F.relu(self.c2(h1)),2)
        h3 = F.max_pooling_2d(F.relu(self.c3(h2)),2)
        h4  = self.l1(h3)
        h5  = self.l2(h4)
        return h5


In [ ]:
model = MyChain()
#optimizer = optimizers.Adam()
optimizer = optimizers.SGD()

optimizer.setup(model)

In [ ]:
losses =[]
batch_num=100
for i in range(2000):
    x,y = get_batch(batch_num)
    x_ = Variable(x.astype(np.float32).reshape((batch_num,1,28,28)))
    t_ = Variable(np.argmax(y,axis=1).astype(np.int32).reshape((batch_num)))
    
    model.zerograds()
    loss=model(x_,t_)
    loss.backward()
    optimizer.update()

    #losses.append(loss.data)
    print loss.data

plt.plot(losses)
plt.yscale('log')

2.31739711761
2.27834939957
2.31021261215
2.28761529922
2.2894051075
2.26229310036
2.25882935524
2.2633497715
2.25219273567
2.25415825844
2.20500969887
2.20990300179
2.2284321785
2.22232174873
2.21196007729
2.1897790432
2.17366456985
2.17402124405
2.17252516747
2.17273163795
2.1409778595
2.15867042542
2.15322709084
2.13625764847
2.12200307846
2.11422133446
2.0901787281
2.12852072716
2.07420992851
2.09447264671
2.05329442024
2.10468769073
2.05862760544
2.07137489319
2.07139253616
2.02251911163
2.0126042366
2.01789617538
2.01789474487
